In [30]:
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [31]:
cfns = pd.read_csv('../data/classifications_cset.csv')

In [32]:
cfns.sample(5)

,classifications.Annotator,classifications.Annotation Status,classifications.Reviewer,classifications.Quality Control,classifications.Full Description,classifications.Short Description,classifications.Beginning Date,classifications.Ending Date,classifications.Location,classifications.Near Miss,...,classifications.Sector of Deployment,classifications.Public Sector Deployment,classifications.Nature of End User,classifications.Level of Autonomy,classifications.Relevant AI functions,classifications.AI Techniques,classifications.AI Applications,classifications.Physical System,classifications.Problem Nature,classifications.Publish
71,2,3. In peer review,6,False,Facebook's automatic language translation soft...,Facebook's automatic language translation soft...,2017-10-15T07:00:00.000Z,2017-10-15T07:00:00.000Z,"Beitar Illit, Israel",Harm caused,...,Information and communication,False,Amateur,High,"[""Perception"",""Cognition"",""Action""]",Facebook language translation software,"[""language translation""]","[""Software only""]","[""Specification""]",False
32,2,6. Complete and final,6,False,There are multiple reports of Amazon Alexa pro...,There are multiple reports of Amazon Alexa pro...,2018-01-01T00:00:00.000Z,2018-01-01T00:00:00.000Z,NaN,Harm caused,...,Information and communication,False,Amateur,Medium,"[""Perception"",""Cognition"",""Action""]",Amazon Alexa; natural language processing; vir...,"[""voice recognition"",""natural language process...","[""Consumer device""]","[""Robustness"",""Assurance""]",True
93,5,3. In peer review,6,False,"In January 2021, HireVue announced it would re...","In January 2021, HireVue removed the controver...",2019-01-01T00:00:00.000Z,1/2021,Global,Unclear/unknown,...,Administrative and support service activities,False,Amateur,Medium,"[""Perception"",""Cognition""]",facial recognition; expression tracking,"[""decision support"",""psychological inference""]","[""Software only""]","[""Specification""]",False
41,5,6. Complete and final,5,False,"From 1982 to 1986, St George's Hospital Medica...","From 1982 to 1986, St George's Hospital Medica...",1982-01-01T00:00:00.000Z,1986-01-01T00:00:00.000Z,"London, England",Harm caused,...,Human health and social work activities,False,Amateur,Medium,"[""Cognition""]",Machine learning,"[""decision support""]","[""Software only""]","[""Specification""]",True
79,5,6. Complete and final,5,False,Scottish soccer team Inverness Caledonian This...,In a Scottish soccer match the AI-enabled ball...,2020-10-24T07:00:00.000Z,2020-10-24T07:00:00.000Z,"Inverness, Scotland, UK",Harm caused,...,"Arts, entertainment and recreation",False,Amateur,High,"[""Perception"",""Cognition"",""Action""]",supervised learning,"[""image classification"",""image recognition""]","[""Consumer device""]","[""Robustness""]",True


In [33]:
# clean up!
cfns.insert(0, 'incident', cfns.index + 1)
cfns.columns = cfns.columns.str.lower().str.replace('classifications.', '').str.replace(' ', '_')
cfns['beginning_date'] = pd.to_datetime(cfns['beginning_date'], utc=True).dt.date
cfns['ending_date'] = pd.to_datetime(cfns['ending_date'], utc=True).dt.date

def parse_list_columns(column):
    if type(column[0]) == str and column[0].startswith('['):
        return column.apply(eval)
    else: return column

cfns = cfns.apply(parse_list_columns)

In [34]:
# cfns.to_csv('../data/classifications_processed.csv', index=False)

In [35]:
cfns.columns

Index(['incident', 'annotator', 'annotation_status', 'reviewer',
       'quality_control', 'full_description', 'short_description',
       'beginning_date', 'ending_date', 'location', 'near_miss',
       'named_entities', 'technology_purveyor', 'intent', 'severity',
       'lives_lost', 'harm_distribution_basis', 'harm_type',
       'infrastructure_sectors', 'financial_cost', 'laws_implicated',
       'ai_system_description', 'data_inputs', 'system_developer',
       'sector_of_deployment', 'public_sector_deployment',
       'nature_of_end_user', 'level_of_autonomy', 'relevant_ai_functions',
       'ai_techniques', 'ai_applications', 'physical_system', 'problem_nature',
       'publish'],
      dtype='object')

In [36]:
cfns.shape

(96, 34)

In [48]:
for col in cfns.columns:
    if type(cfns[col][0]) != list:
        u = cfns[col].unique()
        if len(u) < 10:
            print(col, u)
        else: print(col, len(u))
    else: print(col, 'list')

incident 96
annotator [1 2 3 4 6 5]
annotation_status ['6. Complete and final' '3. In peer review' '4. Peer review complete']
reviewer [5 6 2 3]
quality_control [False  True]
full_description 96
short_description 96
beginning_date 71
ending_date 66
location 58
near_miss ['Harm caused' 'Unclear/unknown' 'Near miss']
named_entities list
technology_purveyor list
intent ['Accident' 'Unclear' 'Deliberate or expected']
severity ['Moderate' 'Critical' 'Severe' 'Minor' 'Negligible' 'Unclear/unknown']
lives_lost [False  True]
harm_distribution_basis list
harm_type list
infrastructure_sectors list
financial_cost [nan 'Short term: $1 trillion   unclear of long-term impact' '$528.00'
 '3.7M Ether ($70M at the time)']
laws_implicated 14
ai_system_description 89
data_inputs 81
system_developer 54
sector_of_deployment 13
public_sector_deployment [False  True]
nature_of_end_user [nan 'Expert' 'Amateur']
level_of_autonomy ['Unclear/unknown' 'High' 'Medium' 'Low' nan]
relevant_ai_functions list
ai_techn

In [53]:
cfns['technology_purveyor']

0                                              [Amazon]
1                                            [Lion Air]
2                               [Uber, Rafaela Vasquez]
3                                                    []
4                                  [Microsoft, Twitter]
                            ...                        
91                                           [Facebook]
92                                          [Deliveroo]
93                                           [HireView]
94    [Houston Independent School District, SAS Inst...
95                                    [Google, YouTube]
Name: technology_purveyor, Length: 96, dtype: object

In [13]:
s = cfns['classifications.Harm Type'].apply(eval).apply(pd.Series).stack().value_counts()
q = np.percentile(s, 50)
s[s > q].index

KeyError: 'classifications.Harm Type'

In [14]:
s

NameError: name 's' is not defined

In [15]:
cfns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   annotator                 96 non-null     int64 
 1   annotation_status         96 non-null     object
 2   reviewer                  96 non-null     int64 
 3   quality_control           96 non-null     bool  
 4   full_description          96 non-null     object
 5   short_description         96 non-null     object
 6   beginning_date            95 non-null     object
 7   ending_date               94 non-null     object
 8   location                  79 non-null     object
 9   near_miss                 96 non-null     object
 10  named_entities            96 non-null     object
 11  technology_purveyor       96 non-null     object
 12  intent                    96 non-null     object
 13  severity                  96 non-null     object
 14  lives_lost                96